In [ ]:
#derive dates of nifti files by matching to dicoms, dataframe queries

In [1]:
import pydicom
import nibabel.nicom.dicomwrappers
import numpy as np
import nibabel as nib
import pandas as pd
import os, glob
from nilearn import image
from nilearn.image import load_img
import dicom2nifti

/var/folders/wf/8pxjw4251t7fl_l85_v1_33r0000gp/T/ipykernel_53570/4022084912.py:2: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.dicomwrappers


In [2]:
img = nibabel.nicom.dicomwrappers.wrapper_from_file('/Volumes/Cingulate/Cingulotomy/original/DICOM/3523469/8461478/MR/Axial_FSE_T2/469466715.dcm')
print(img.dcm_data.AcquisitionDate)
print(img.dcm_data.PatientBirthDate)

20041015
19731026


In [478]:
def find_files_with_suffix(root_path, suffix):
    nii_gz_files = []
    for subdir, dirs, files in os.walk(root_path):
        for file in files:
            if file.endswith(suffix):
                full_path = os.path.join(subdir, file)
                nii_gz_files.append(full_path)

    return nii_gz_files

def find_files_with_suffix_dcm(root_path, suffix):
    nii_gz_files = []
    for subdir, dirs, files in os.walk(root_path):
        for file in files:
            if file.endswith(suffix):
                full_path = os.path.join(subdir, file)
                nii_gz_files.append(full_path)
                break

    return nii_gz_files

def match_pairs(nii_files, dicom_files):
    matching_pair_nii = []
    matching_pair_dicom = []
    for nii_data in nii_files:
        nii_file = nib.load(nii_data)
        nii_info = extract_nii_info(nii_file)

        len_z = len(dicom_files)
        for dcm in dicom_files:
            dicom_data = pydicom.dcmread(dcm)
            dicom_info = extract_dicom_info(dicom_data)

            if compare_headers(nii_info, dicom_info):
                matching_pair_nii.append(nii_data)
                matching_pair_dicom.append(dcm)
            else:
                print(nii_info['dim'], dicom_info['dim'])
                print(nii_info['pixdim'], dicom_info['pixdim'])
    return matching_pair_nii, matching_pair_dicom

def extract_nii_info(nii_data):
    header = nii_data.header
    info = {
        'dim': header['dim'][1:3], #could be [1:4] if z plane included 
        'pixdim': header['pixdim'][1:3],
    }
    return info

def extract_dicom_info(dicom_data):
    info = {
        'dim': np.array(dicom_data.pixel_array.shape),
        'pixdim': np.array([dicom_data.PixelSpacing[0], dicom_data.PixelSpacing[1]])#, dicom_data.SliceThickness]),
    }
    return info

def compare_headers(nii_info, dicom_info):
    return np.array_equal(nii_info['dim'], dicom_info['dim']) and np.array_equal(nii_info['pixdim'], dicom_info['pixdim'])

In [672]:
dcm_dirs = '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-3526851/DICOM'
dicom_files = find_files_with_suffix_dcm(dcm_dirs,'.dcm')
nii_dirs = '/Volumes/Cingulate/Cingulotomy/original/rawdata/sub-3526851'
nii_files = find_files_with_suffix(nii_dirs,'.nii.gz')

In [669]:
retain = [i for i in dicom_files if '/MR/' in i]
matching_nii, matching_dcm = match_pairs(nii_files, retain)

[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [256 256]
[0.937493  1.4996758] [0.9766 0.9766]
[256 124] [256 256]
[0.937493  1.4996758] [0.9766 0.9766]
[256 124] [512 512]
[0.937493  1.4996758] [0.4883 0.4883]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [512 512]
[0.937493  1.4996758] [0.625 0.625]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [64 64]
[0.937493  1.4996758] [4.6875 4.6875]
[256 124] [256 256   3]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] [256 256]
[0.937493  1.4996758] [0.859375 0.859375]
[256 124] [256 256]
[0.937493  1.4996758] [0.9375 0.9375]
[256 124] 

In [670]:
matching_dcm

['/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/DWI/469441994.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/t1_sag/469436238.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/Axial_SPGR/469441934.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/ZERO_B/469442127.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/DIFFUSION/469438165.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/ADC/469442178.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/3_pl_T2__FGRE_S/469435581.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/FA/469442193.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/sourcedata/sub-4459639/DICOM/11182859/MR/EXP/469442150.dcm',
 '/Volumes/Cingulate/Cingulotomy/original/s

In [462]:
for i in matching_dcm:
    dicoom = nibabel.nicom.dicomwrappers.wrapper_from_file(i)
    date = dicoom.dcm_data.AcquisitionDate

df = pd.DataFrame(
    {'ID': dicoom.dcm_data.PatientID,
     'surgery_type': 'cing,sct',
     'date': dicoom.dcm_data.AcquisitionDate,
     'matching_nii': matching_nii,
     'matching_dcm': matching_dcm
    })

df.to_csv('/Volumes/Cingulate/Cingulotomy/segmentations/date_csvs/sub-3523469.csv')

In [654]:
#reorganize dates to match cingulotomy csv
df = pd.read_csv('/Volumes/Cingulate/Cingulotomy/segmentations/date_csvs/dicom_nii_dates.csv')
for i in df['date']:
    i = str(i)
    year = i[0:4]
    month = i[4:6]
    day = i[6:9]
    new_date = f'{month}/{day}/{year}'

In [ ]:
#add y and m onto dates
df = pd.read_csv('/Users/jp1590/Desktop/dates.csv')
for i in df['SCT_Time_From_OR2_Y']:
    print(f'{i}y')

for j in df['SCT_Time_From_OR2_M']:
    print(f'{j}m')

#add strings together in columns
df = pd.read_csv('/Users/jp1590/Desktop/dates.csv')
df['Cing_Time_From_OR1'] = df['Cing_Time_From_OR1_Y'].astype(str) + ' ' + df['Cing_Time_From_OR1_M'].astype(str)
df['Cing_Time_From_OR2'] = df['Cing_Time_From_OR2_Y'].astype(str) + ' ' + df['Cing_Time_From_OR2_M'].astype(str)
df['SCT_Time_From_OR1'] = df['SCT_Time_From_OR1_Y'].astype(str) + ' ' + df['SCT_Time_From_OR1_M'].astype(str)
df['SCT_Time_From_OR2'] = df['SCT_Time_From_OR2_Y'].astype(str) + ' ' + df['SCT_Time_From_OR2_M'].astype(str)
df.to_csv('/Users/jp1590/Desktop/dates.csv')

In [55]:
#check if one value in one dataframe is in another
ps_df = pd.read_csv('/Users/jp1590/Desktop/Processed Subjects.csv')
id_df = pd.read_csv('/Users/jp1590/Desktop/mrn-id.csv')

#prints out a separate column if it is in (1 = yes, 0 = no)
id_df = id_df.assign(result=id_df['MRN'].isin(ps_df['MRN']).astype(int))
id_df.to_csv('/Users/jp1590/Desktop/matched-mrns.csv')

In [41]:
files = '/Volumes/Cingulate/GDNF/Follow up MRI/0015-Subject-GD-24'
for dirpath, dirnames, filenames in os.walk(files):
    if any(filename.endswith('.dcm') for filename in filenames):
        dicom2nifti.convert_directory(f'{dirpath}','/Volumes/Cingulate/GDNF/GD24_nii')

Slice increment not consistent through all slices
---------------------------------------------------------
[-115.81310247 -104.02644753  -35.59370182] [ 0.01436885  0.14936733 -1.99436188]
[-115.8418402  -104.3251822   -31.60497616] [ 0.02873772  0.29873466 -3.98872566]
Instance Number: 976
---------------------------------------------------------
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dicom2nifti/convert_dir.py", line 85, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dicom2nifti/convert_dicom.py", line 124, in dicom_array_to_nifti
    results = convert_philips.dicom_to_nifti(dicom_list, output_file)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dicom2nifti/

In [76]:
#rename nii files with date and type of scan
for i in os.listdir('/Volumes/Cingulate/GDNF/GD21_nii'):
    if os.path.exists(f'/Volumes/Cingulate/GDNF/Follow up MRI/0015-Subject-GD-21/0015-50/{i[:-7]}'):
        img = nibabel.nicom.dicomwrappers.wrapper_from_file(f'/Volumes/Cingulate/GDNF/Follow up MRI/0015-Subject-GD-21/0015-50/{i[:-7]}/1.dcm')
        os.rename(f'/Volumes/Cingulate/GDNF/GD21_nii/{i}',f'/Volumes/Cingulate/GDNF/GD21_nii/{img.dcm_data.AcquisitionDate}_{img.dcm_data.SeriesDescription}.nii.gz')

In [151]:
main_folder_path = '/Volumes/Cingulate/GDNF/GD24/15829034'

# Iterate through subfolders in the main folder
for subfolder in os.listdir(main_folder_path):
    subfolder_path = os.path.join(main_folder_path, subfolder)

    # Check if the item is a directory
    if os.path.isdir(subfolder_path):
        # Get a list of files in the subfolder
        files = os.listdir(subfolder_path)

        # Check if there are any files in the subfolder
        if files:
            # Get the path of the first file in the subfolder
            first_file_path = os.path.join(subfolder_path, files[0])
            img = nibabel.nicom.dicomwrappers.wrapper_from_file(first_file_path)
            new_subfolder_path = os.path.join(main_folder_path, f'{img.dcm_data.AcquisitionDate}_{img.dcm_data.SeriesDescription}')
            if not os.path.exists(new_subfolder_path):
                os.rename(subfolder_path, new_subfolder_path)

In [43]:
from pydicom import dcmread
ids = []
dates = {}
for dirpath, dirnames, filenames in os.walk(path):
    processed_folder = False
    for filename in filenames:
        if filename.endswith(('.dcm', '.IMA')):
            file_path = os.path.join(dirpath, filename)
            try:
                ds = dcmread(file_path)
                
                # Check if the folder has already been processed
                if not processed_folder:
                    ids.append(dirpath[27:34])
                    dates[dirpath[27:34]] = ds.get("AcquisitionDate", "Unknown")
                    processed_folder = True
            
            except errors.InvalidDicomError:
                print(f"Skipping non-DICOM file: {file_path}")
                continue

In [41]:
dates #old

{'NCH001/': '20211130',
 'NCH002/': '20220119',
 'NCH003/': '20220412',
 'NCH004/': '20220726',
 'NCH005/': '20221025',
 'OSU001/': '20201013',
 'POL009/': '20190416',
 'POL010/': '20190514',
 'POL011/': '20190716',
 'POL017/': '20191210'}

In [44]:
dates

{'POL018/': '20191217',
 'POL019/': '20200218',
 'POL020/': '20200220',
 'UCSF001': '20170110',
 'UCSF002': '20170502',
 'UCSF006': '20190814'}